In [1]:
import os
import pandas
import tqdm
import functools
from sklearn import preprocessing

In [2]:
source = '../resource/kaggle/sample/csv'
table = {
    'article':pandas.read_csv(os.path.join(source, 'articles.csv'), dtype=str),
    'customer':pandas.read_csv(os.path.join(source, 'customers.csv'), dtype=str),
    'transaction':pandas.read_csv(os.path.join(source, 'transactions_train.csv'), dtype=str),
    'submission':pandas.read_csv(os.path.join(source, 'sample_submission.csv'), dtype=str)
}

# Missing

In [3]:
table['article']['detail_desc'].fillna('miss_detail_desc', inplace=True)
table['customer']['FN'].fillna("0.0", inplace=True)
table['customer']['Active'].fillna("0.0", inplace=True)
table['customer']['fashion_news_frequency'].fillna("NONE", inplace=True)
table['customer']['age'].fillna("-1", inplace=True)
table['customer']['postal_code'].fillna('miss_postal_code', inplace=True)

# Numeralization

In [4]:
table['article']["article_label"] = table['article']['article_id']
for key in table['article']:

    if(key=='article_id'): continue

    skip = 10
    engine = preprocessing.LabelEncoder()
    engine.fit(table['article'][key])
    value = engine.transform(table['article'][key])
    table['article'][key] = value + skip
    continue

In [5]:
table['customer']['FN'] = table['customer']['FN'].astype(float)
table['customer']['Active'] = table['customer']['Active'].astype(float)
table['customer']['age'] = table['customer']['age'].astype(float) / 100
for key in table['customer']:

    if(key in ['customer_id', "FN", "Active", 'age']): continue

    skip = 10
    engine = preprocessing.LabelEncoder()
    engine.fit(table['customer'][key])
    value = engine.transform(table['customer'][key])
    table['customer'][key] = value + skip
    continue

In [6]:
table['transaction']['t_dat'] = pandas.to_datetime(table['transaction']['t_dat'])
value = table['transaction']['price'].astype(float).copy()
value = value / value.max()
table['transaction']['price'] = value
for key in table['transaction']:
    
    if(key in ['t_dat', "customer_id", "article_id", 'price']): continue

    skip = 10
    engine = preprocessing.LabelEncoder()
    engine.fit(table['transaction'][key])
    value = engine.transform(table['transaction'][key])
    table['transaction'][key] = value + skip
    continue

# Feature with user base

In [7]:
paste = lambda x: " ".join(x)

In [13]:
sequence = []

In [14]:
table['transaction']['length'] = table['transaction']['customer_id']
sequence += [table['transaction'].groupby('customer_id').count().reset_index()[['customer_id', 'length']]]

In [15]:
table['transaction']['price'] = table['transaction']['price'].astype(str)
table['transaction']['sales_channel_id'] = table['transaction']['sales_channel_id'].astype(str)
sequence += [table['transaction'].groupby(['customer_id', 't_dat'])['price'].apply(paste).reset_index().groupby(['customer_id'])['price'].apply(paste).reset_index()]
sequence += [table['transaction'].groupby(['customer_id', 't_dat'])['sales_channel_id'].apply(paste).reset_index().groupby(['customer_id'])['sales_channel_id'].apply(paste).reset_index()]

In [16]:
for key in tqdm.tqdm(table['article'].keys()):

    if(key=='article_id'): continue

    table['article'][key] = table['article'][key].astype(str)
    selection = table['transaction'][['t_dat', "customer_id", "article_id"]].copy()
    selection = pandas.merge(selection, table['article'][["article_id", key]], on="article_id", how='inner')
    sequence += [selection.groupby(['customer_id', 't_dat'])[key].apply(paste).reset_index().groupby(['customer_id'])[key].apply(paste).reset_index()]
    continue

100%|██████████| 26/26 [01:45<00:00,  4.04s/it]


In [17]:
merge = lambda x,y: pandas.merge(left=x, right=y, on='customer_id', how='inner')
sequence = functools.reduce(merge, sequence)
sequence.head(4)

,customer_id,length,price,sales_channel_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,...,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,article_label
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,7,0.11458452722063038 0.0429512893982808 0.05727...,11 11 11 11 11 11 11,8739 4298 8662 1593 11113 11593 4298,2043 8449 9458 11138 5775 5296 8449,38 30 24 17 17 17 30,76 31 74 85 85 85 31,14 13 15 15 15 15 13,26 33 20 26 26 26 33,...,13 13 10 13 10 10 13,14 14 16 14 16 16 14,11 11 10 11 10 10 11,11 11 12 11 12 12 11,44 44 12 44 12 10 44,22 22 54 22 54 62 22,20 21 18 12 12 12 21,22 13 11 17 17 17 13,7849 8521 7678 4456 5522 4401 8521,17873 9592 17733 3982 21753 22552 9592
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,3,0.08593123209169053 0.051547277936962746 0.085...,11 11 11,10859 2083 10859,795 9752 795,41 60 41,98 21 98,22 22 22,26 26 26,...,11 11 11,17 17 17,10 10 10,12 12 12,51 51 51,56 56 56,24 24 24,28 28 28,4829 4843 4829,21316 4955 21316
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,23,0.025759312320916907 0.028624641833810888 0.02...,11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 1...,1219 12274 12274 12274 1219 3266 1295 5528 164...,5483 12835 12835 12835 5485 12674 5511 10033 2...,17 17 17 17 17 60 52 44 44 44 60 86 98 103 12 ...,85 85 85 85 85 21 68 99 99 99 21 44 12 71 37 7...,15 15 15 15 15 22 18 22 22 22 22 10 19 19 19 1...,20 20 20 20 20 26 24 20 26 26 26 26 26 26 26 2...,...,13 10 10 10 13 11 11 13 11 11 11 13 12 12 13 1...,14 16 16 16 14 17 17 14 17 17 17 14 15 15 14 1...,11 10 10 10 11 10 10 11 10 10 10 11 10 10 11 1...,11 12 12 12 11 12 12 11 12 12 12 11 12 12 11 1...,48 12 12 12 48 51 52 42 51 51 51 43 53 53 43 4...,25 54 54 54 25 56 57 21 56 56 56 19 59 59 19 1...,12 29 29 29 12 24 24 24 24 24 24 26 27 27 27 2...,17 24 24 24 17 28 28 28 28 28 28 10 20 20 20 2...,4500 11669 11669 11669 4500 1370 7013 10724 13...,3153 23662 23663 23660 3152 7536 3315 11742 40...
3,00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d...,1,0.05727793696275071,11,2701,9886,62,49,15,26,...,14,18,13,13,23,48,11,15,11073,6282


In [18]:
table['feature'] = pandas.merge(left=table['customer'], right=sequence, on='customer_id', how='outer')

In [20]:
folder = '../resource/preprocess/sample/csv/'
os.makedirs(folder, exist_ok=True)
table['feature'].dropna().to_csv(os.path.join(folder, "feature(train).csv"), index=False)
table['feature'].fillna("").to_csv(os.path.join(folder, "feature(all).csv"), index=False)

---